In [1]:
import json
import base64
import qrcode
import requests
import datetime
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.message import Attachment, AttachmentDataJson, AttachmentDataLinks
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID

from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [2]:
secrets_resolver = SecretsResolverDemo()

In [3]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
             didcomm_services = []
#                 DIDCommService(
#                     id = s.id,
#                     service_endpoint = s.service_endpoint,
#                     routing_keys = s.routing_keys,
#                     accept = s.accept
#                 )
#                 for s in did_doc.service
#                 if isinstance(s, DIDCommServicePeerDID)
#            ] if did_doc.service else []
        )

In [4]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="#didcommmessaging-0",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


In [28]:
async def poll_response():
    alice_status_check = Message(
        id = str(uuid.uuid4()),
        type = "https://didcomm.org/messagepickup/3.0/status-request",
        body = {}
    )
    alice_status_check_packed_msg = await pack_encrypted(
        resolvers_config = ResolversConfig(
            secrets_resolver = secrets_resolver,
            did_resolver = DIDResolverPeerDID()
        ),
        message = alice_status_check,
        to = mediator_did,
        frm = alice_did_for_mediator,
        sign_frm = None,
        pack_config = PackEncryptedConfig(protect_sender_id=False)
    )
    headers = {"Content-Type": "application/didcomm-encrypted+json"}
    resp3 = requests.post(mediator_endpoint, headers=headers, data = alice_status_check_packed_msg.packed_msg)
    mediator_unpack_status = await unpack(
        resolvers_config=ResolversConfig(
            secrets_resolver=secrets_resolver,
            did_resolver=DIDResolverPeerDID()
        ),
        packed_msg= resp3.json()
    )
    print("Messages in Mediator queue: " + str(mediator_unpack_status.message.body["message_count"]))
    if mediator_unpack_status.message.body["message_count"]>0:
        alice_delivery_request = Message(
            id = str(uuid.uuid4()),
            type = "https://didcomm.org/messagepickup/3.0/delivery-request",
            body = {"limit": 1}
        )
        alice_delivery_request_packed_msg = await pack_encrypted(
            resolvers_config = ResolversConfig(
                secrets_resolver = secrets_resolver,
                did_resolver = DIDResolverPeerDID()
            ),
            message = alice_delivery_request,
            to = mediator_did,
            frm = alice_did_for_mediator,
            sign_frm = None,
            pack_config = PackEncryptedConfig(protect_sender_id=False)
        )
        headers = {"Content-Type": "application/didcomm-encrypted+json"}
        resp4 = requests.post(mediator_endpoint, headers=headers, data = alice_delivery_request_packed_msg.packed_msg)
        mediator_delivery = await unpack(
            resolvers_config=ResolversConfig(
                secrets_resolver=secrets_resolver,
                did_resolver=DIDResolverPeerDID()
            ),
            packed_msg= resp4.json()
        )
        bob_packed_msg = mediator_delivery.message.attachments[0].data.json
        msg_id = mediator_delivery.message.attachments[0].id
        bob_msg = await unpack(
            resolvers_config=ResolversConfig(
                secrets_resolver=secrets_resolver,
                did_resolver=DIDResolverPeerDID()
            ),
            packed_msg= bob_packed_msg
        )

        alice_ack = Message(
        id = str(uuid.uuid4()),
        type = "https://didcomm.org/messagepickup/3.0/messages-received",
        body = {"message_id_list": [msg_id]}
        )
        alice_ack_packed_msg = await pack_encrypted(
            resolvers_config = ResolversConfig(
                secrets_resolver = secrets_resolver,
                did_resolver = DIDResolverPeerDID()
            ),
            message = alice_ack,
            to = mediator_did,
            frm = alice_did_for_mediator,
            sign_frm = None,
            pack_config = PackEncryptedConfig(protect_sender_id=False)
        )
        headers = {"Content-Type": "application/didcomm-encrypted+json"}
        resp5 = requests.post(mediator_endpoint, headers=headers, data = alice_ack_packed_msg.packed_msg)
        mediator_ack_status = await unpack(
            resolvers_config=ResolversConfig(
                secrets_resolver=secrets_resolver,
                did_resolver=DIDResolverPeerDID()
            ),
            packed_msg= resp5.json()
        )
        print(f'message {msg_id} deleted')
#         print(bob_msg.message.attachments[0].data.json)
        return bob_msg

### Alice get OOB from Mediator

In [10]:
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text

In [11]:
print(oob_url)

https://mediator.rootsid.cloud?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiOWQ5ZjdhMzYtYmYzYS00YWViLWE5MTItMDExZjM3MTc4NzM3IiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNtczU1NVloRnRobjFXVjhjaURCcFptODZoSzl0cDgzV29qSlVteFBHazFoWi5WejZNa21kQmpNeUI0VFM1VWJiUXc1NHN6bTh5dk1NZjFmdEdWMnNRVllBeGFlV2hFLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwibGFiZWwiOiJNZWRpYXRvciIsImFjY2VwdCI6WyJkaWRjb21tL3YyIl19fQ


In [12]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
alice_did_for_mediator = await create_peer_did(1,1)
print("Alice's DID:", alice_did_for_mediator)

Alice's DID: did:peer:2.Ez6LSc4Zn6PvCmWdP4nEv4agqB6gQ2KEcZHgrk44W2woTpK8p.Vz6MkhMFCouiXTJRsMjZQVHHCaaEMEfY9JkyPU3r1EhEiqC1G


### Alice request mediate

In [13]:
alice_mediate_grant = Message(
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/coordinate-mediation/2.0/mediate-request",
    body = {}
)

In [14]:
alice_mediate_grant_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_mediate_grant,
    frm = alice_did_for_mediator,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending to Mediator

In [15]:
mediator_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_endpoint)

https://mediator.rootsid.cloud


In [16]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, json = json.loads(alice_mediate_grant_packed.packed_msg))


In [17]:
print(resp.json())

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTNCQ1ZXcHdaVTFTTVdjNVZYRm5VbWswT0VWaFkyOU9ibmx6UjFOQ2JXOUlXbVJrVUd0eVYxcHFNVGhOTGxaNk5rMXJhRkZqUkRsd1RVaFdkWEJIYUdSWFUyMUVhWFZTWkVwUWMyTlFSRUZpYTFGM056aFZhMVk1VURsTk5rWXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3eU1XeGFSMnhvWkVjNWVVeHVTblppTTFKNllWZFJkVmt5ZUhaa1YxRnBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGZ3dJelpNVTNCQ1ZXcHdaVTFTTVdjNVZYRm5VbWswT0VWaFkyOU9ibmx6UjFOQ2JXOUlXbVJrVUd0eVYxcHFNVGhOIiwiYXB2IjoiTVZyb0NJQ2g4alRxcEpfa3pVSVNENlFvQUI3XzJsMmlNUmQ4TXFDZGY2QSIsInNraWQiOiJkaWQ6cGVlcjoyLkV6NkxTcEJVanBlTVIxZzlVcWdSaTQ4RWFjb05ueXNHU0Jtb0haZGRQa3JXWmoxOE0uVno2TWtoUWNEOXBNSFZ1cEdoZFdTbURpdVJkSlBzY1BEQWJrUXc3OFVrVjlQOU02Ri5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTc

### Get mediator grant response

In [18]:
mediator_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg.message.type)
print("Routing Key: " + mediator_unpack_msg.message.body["routing_did"])
print("Mediator new DID: " + mediator_unpack_msg.message.from_prior.sub)
# mediator rotated did
mediator_routing_key = mediator_unpack_msg.message.body["routing_did"]
mediator_did = mediator_unpack_msg.message.from_prior.sub

https://didcomm.org/coordinate-mediation/2.0/mediate-grant
Routing Key: did:peer:2.Ez6LScaajHbVyFKq2QX1z4g579mRCRAzmEsS4tCepafKiZbJ9.Vz6MkqyvfBunRfGYx5xjbEoKaXEV4ne3WXDWDUsRKdpEx1P7w.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0
Mediator new DID: did:peer:2.Ez6LSpBUjpeMR1g9UqgRi48EacoNnysGSBmoHZddPkrWZj18M.Vz6MkhQcD9pMHVupGhdWSmDiuRdJPscPDAbkQw78UkV9P9M6F.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0


### Keylist update

Alice create a new DID to share with  other peers using mediator routing keys 

In [19]:
alice_did_new = await create_peer_did(1, 1, service_endpoint=mediator_endpoint, service_routing_keys=[mediator_routing_key])
# alice_did_for_bob = await create_peer_did(1, 1, service_endpoint=mediator_routing_key)

print("Alice's DID for Bob:", alice_did_new)

Alice's DID for Bob: did:peer:2.Ez6LSgNmZmqQFzvbVaE2mKYcF1TFrU89c88seVEwqL3FwENAc.Vz6MkqMG9iqP9iAk6B2qbLHs4yLavCXm54cPgW6GBdXPRT9gF.SeyJpZCI6IiNkaWRjb21tbWVzc2FnaW5nLTAiLCJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9tZWRpYXRvci5yb290c2lkLmNsb3VkIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2NhYWpIYlZ5RktxMlFYMXo0ZzU3OW1SQ1JBem1Fc1M0dENlcGFmS2laYko5LlZ6Nk1rcXl2ZkJ1blJmR1l4NXhqYkVvS2FYRVY0bmUzV1hEV0RVc1JLZHBFeDFQN3cuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyMWxaR2xoZEc5eUxuSnZiM1J6YVdRdVkyeHZkV1FpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIl19


Alice create and send the keylist update message

In [20]:
alice_keylist_update = Message(
    id = "unique-id-293e9a922efff",
    type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
    body = {
        "updates":[
        {
            "recipient_did": alice_did_new,
            "action": "add"
        }
    ]
    }
)

In [21]:
alice_keylist_update_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_keylist_update,
    frm = alice_did_for_mediator,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [22]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_keylist_update_packed_msg.packed_msg)

In [23]:
mediator_unpack_msg2 = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg2.message.type)
print(mediator_unpack_msg2.message.body)


https://didcomm.org/coordinate-mediation/2.0/keylist-update-response
{'updated': [{'recipient_did': 'did:peer:2.Ez6LSgNmZmqQFzvbVaE2mKYcF1TFrU89c88seVEwqL3FwENAc.Vz6MkqMG9iqP9iAk6B2qbLHs4yLavCXm54cPgW6GBdXPRT9gF.SeyJpZCI6IiNkaWRjb21tbWVzc2FnaW5nLTAiLCJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9tZWRpYXRvci5yb290c2lkLmNsb3VkIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2NhYWpIYlZ5RktxMlFYMXo0ZzU3OW1SQ1JBem1Fc1M0dENlcGFmS2laYko5LlZ6Nk1rcXl2ZkJ1blJmR1l4NXhqYkVvS2FYRVY0bmUzV1hEV0RVc1JLZHBFeDFQN3cuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyMWxaR2xoZEc5eUxuSnZiM1J6YVdRdVkyeHZkV1FpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIl19', 'action': 'add', 'result': 'success'}]}


### Alice can now send her DID message to Nessus

In [24]:
#get nessus
nessus_invitation = json.loads(requests.get('http://nessus-tech.io:9100/invitation?inviter=Government&method=peer').text)
nessus_did_doc = json.loads(peer_did.resolve_peer_did(nessus_invitation["from"]))
nessus_did_doc

{'id': 'did:peer:2.Ez6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL.Vz6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ',
 'authentication': [{'id': 'did:peer:2.Ez6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL.Vz6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ#6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE',
   'type': 'Ed25519VerificationKey2020',
   'controller': 'did:peer:2.Ez6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL.Vz6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ',
   'publicKeyMultibase': 'z6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE'}],
 'keyAgreement': [{'id': 'did:peer:2.Ez6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL.Vz6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ#6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL',
   'type': '

In [32]:
ping_msg = Message(
    body = { "response_requested": True , 'suggested_label': "RootsJNNotebook" },
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/trust-ping/2.0/ping",
    frm = alice_did_new,
    to = [nessus_invitation["from"]],
)

tony_trust_ping_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = ping_msg,
    frm = alice_did_new,
    to = nessus_invitation['from'],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
trust_ping_did_doc = json.loads(peer_did.resolve_peer_did(nessus_invitation["from"]))
trust_ping_endpoint = trust_ping_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(trust_ping_endpoint, headers=headers, json = json.loads(tony_trust_ping_packed.packed_msg))
resp.text

'{}'

### Alice check status of queued messages from the Mediator

In [33]:
res = await poll_response()

Messages in Mediator queue: 1
message 6435ec0efb5566e28c19e283 deleted


In [37]:
res.message.as_dict()

{'id': '3298ca6b-1df6-41fd-95b3-e75168ce59e5',
 'type': 'https://didcomm.org/trust-ping/2.0/ping-response',
 'body': {'comment': 'Pong from Government', 'suggested_label': 'Government'},
 'to': ['did:peer:2.Ez6LSgNmZmqQFzvbVaE2mKYcF1TFrU89c88seVEwqL3FwENAc.Vz6MkqMG9iqP9iAk6B2qbLHs4yLavCXm54cPgW6GBdXPRT9gF.SeyJpZCI6IiNkaWRjb21tbWVzc2FnaW5nLTAiLCJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9tZWRpYXRvci5yb290c2lkLmNsb3VkIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2NhYWpIYlZ5RktxMlFYMXo0ZzU3OW1SQ1JBem1Fc1M0dENlcGFmS2laYko5LlZ6Nk1rcXl2ZkJ1blJmR1l4NXhqYkVvS2FYRVY0bmUzV1hEV0RVc1JLZHBFeDFQN3cuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyMWxaR2xoZEc5eUxuSnZiM1J6YVdRdVkyeHZkV1FpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIl19'],
 'created_time': 1681255438,
 'expires_time': 1681341838,
 'thid': 'a8b0cc9d-9e4f-4c3c-a03c-df6787796e40',
 'from': 'did:peer:2.Ez6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL.Vz6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25

In [38]:

tony_basic_message = Message(
    id = str(uuid.uuid4()),
    thid = str(uuid.uuid4()),
    type="https://didcomm.org/basicmessage/2.0/message",
    body={"content": f'Nessus, this is the Notebook(not the movie) {str(uuid.uuid4())[:10]}'},
    created_time= int(datetime.datetime.now().timestamp()),
    to = [nessus_invitation['from']],
        frm = alice_did_new,
)

tony_basic_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = tony_basic_message,
    frm = alice_did_new,
    to = nessus_invitation['from'],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

trust_ping_did_doc = json.loads(peer_did.resolve_peer_did(nessus_invitation["from"]))
trust_ping_endpoint = trust_ping_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(trust_ping_endpoint, headers=headers, json = json.loads(tony_basic_message_packed.packed_msg))
resp.text

'{}'

# Propose Credential 

In [39]:

tony_basic_message = Message(
    id = str(uuid.uuid4()),
    thid = str(uuid.uuid4()),
    type="https://didcomm.org/issue-credential/3.0/propose-credential",
    body={
    "goal_code": "Issue Passport Credential",
    "credential_preview": [
      {
        "name": "givenName",
        "value": "Alex"
      },
      {
        "name": "familyName",
        "value": "Andrei"
      },
      {
        "name": "citizenship",
        "value": "US"
      }
    ]
  },
    created_time= int(datetime.datetime.now().timestamp()),
    to = [nessus_invitation['from']],
    frm = alice_did_new,
    attachments=[Attachment(
            data=AttachmentDataJson(json={
          "type": [
            "VerifiableCredential",
            "Passport"
          ],
          "@context": [
            "https://www.w3.org/2018/credentials/v1"
          ],
          "id": "urn:uuid:ee9205ac-0f67-4c22-b961-968c4a5db2c0",
          "issuer": "did:key:z6Mkoe1KHk9aEJeTbWtC6AQTpX7FeUMNqSSVubVgY2M1X9st",
          "issuanceDate": "2023-04-06T10:58:22.335067Z",
          "credentialSchema": {
            "id": "class:nessus/schema/PassportSchema-draft-01.json",
            "type": "JsonSchemaValidator2018"
          },
          "credentialSubject": {
            "id": "did:key:z6Mkrea1ndAivx7y7iXVgSBK98BLYmBm7RPdfEtwRN3MSVfj",
            "givenName": "Alex",
            "familyName": "Andrei",
            "citizenship": "US"
          }
        }),
        media_type="application/json",
        format="https://www.w3.org/TR/vc-data-model/"
    )])

tony_basic_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = tony_basic_message,
    frm = alice_did_new,
    to = nessus_invitation['from'],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

trust_ping_did_doc = json.loads(peer_did.resolve_peer_did(nessus_invitation["from"]))
trust_ping_endpoint = trust_ping_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(trust_ping_endpoint, headers=headers, json = json.loads(tony_basic_message_packed.packed_msg))
resp.text

'{}'

In [40]:
res = await poll_response()
res

Messages in Mediator queue: 1
message 6435ec6afb5566e28c19e284 deleted


UnpackResult(message=Message(id='8bad884d-a36c-4602-a7ec-1db284fd73eb', type='https://didcomm.org/issue-credential/3.0/offer-credential', body={'goal_code': 'Issue Passport Credential', 'credential_preview': [{'name': 'id', 'value': 'did:peer:2.Ez6LSgNmZmqQFzvbVaE2mKYcF1TFrU89c88seVEwqL3FwENAc.Vz6MkqMG9iqP9iAk6B2qbLHs4yLavCXm54cPgW6GBdXPRT9gF.SeyJpZCI6IiNkaWRjb21tbWVzc2FnaW5nLTAiLCJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9tZWRpYXRvci5yb290c2lkLmNsb3VkIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2NhYWpIYlZ5RktxMlFYMXo0ZzU3OW1SQ1JBem1Fc1M0dENlcGFmS2laYko5LlZ6Nk1rcXl2ZkJ1blJmR1l4NXhqYkVvS2FYRVY0bmUzV1hEV0RVc1JLZHBFeDFQN3cuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyMWxaR2xoZEc5eUxuSnZiM1J6YVdRdVkyeHZkV1FpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIl19'}, {'name': 'givenName', 'value': 'Alex'}, {'name': 'familyName', 'value': 'Andrei'}, {'name': 'citizenship', 'value': 'US'}]}, frm='did:peer:2.Ez6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL.Vz6MkkL7EaycmQTpdiodvEfQipKmjy2

In [42]:
res.message.attachments[0].as_dict()

{'data': {'json': {'type': ['VerifiableCredential', 'Passport'],
   '@context': ['https://www.w3.org/2018/credentials/v1',
    'https://w3id.org/security/suites/jws-2020/v1'],
   'id': 'urn:uuid:a681b5ee-5ffa-4ac0-8850-8ad36809c71f',
   'issuer': 'did:peer:2.Ez6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL.Vz6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ',
   'issuanceDate': '2023-04-11T23:25:28.395251235Z',
   'credentialSchema': {'id': 'class:nessus/schema/PassportSchema-draft-01.json',
    'type': 'JsonSchemaValidator2018'},
   'credentialSubject': {'id': 'did:peer:2.Ez6LSgNmZmqQFzvbVaE2mKYcF1TFrU89c88seVEwqL3FwENAc.Vz6MkqMG9iqP9iAk6B2qbLHs4yLavCXm54cPgW6GBdXPRT9gF.SeyJpZCI6IiNkaWRjb21tbWVzc2FnaW5nLTAiLCJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9tZWRpYXRvci5yb290c2lkLmNsb3VkIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2NhYWpIYlZ5RktxMlFYMXo0ZzU3OW1SQ1JBem1Fc1M0dENlcGFmS2laYko5LlZ6Nk1rcXl2ZkJ1blJmR1l4NXhqYkVvS2FYRVY0bmUzV1hE

# Request Credential

In [43]:

tony_basic_message = Message(
    id = str(uuid.uuid4()),
    type="https://didcomm.org/issue-credential/3.0/request-credential",
    body={
    "goal_code": "Issue Passport Credential"
  },
    created_time= int(datetime.datetime.now().timestamp()),
    to = [nessus_invitation['from']],
    frm = alice_did_new,
    attachments=[Attachment(
            data=AttachmentDataJson(json={
          "type": [
            "VerifiableCredential",
            "Passport"
          ],
          "@context": [
            "https://www.w3.org/2018/credentials/v1",
            "https://w3id.org/security/suites/jws-2020/v1"
          ],
          "id": "urn:uuid:1cece1f8-6a68-4361-8823-08802b56ede8",
          "issuer": "did:key:z6Mkoe1KHk9aEJeTbWtC6AQTpX7FeUMNqSSVubVgY2M1X9st",
          "issuanceDate": "2023-04-06T10:58:22.435583926Z",
          "credentialSchema": {
            "id": "class:nessus/schema/PassportSchema-draft-01.json",
            "type": "JsonSchemaValidator2018"
          },
          "credentialSubject": {
            "id": "did:key:z6Mkrea1ndAivx7y7iXVgSBK98BLYmBm7RPdfEtwRN3MSVfj",
            "givenName": "Alex",
            "familyName": "Andrei",
            "citizenship": "US"
          },
          "proof": {
            "type": "JsonWebSignature2020",
            "creator": "did:key:z6Mkoe1KHk9aEJeTbWtC6AQTpX7FeUMNqSSVubVgY2M1X9st",
            "created": "2023-04-06T10:58:22Z",
            "proofPurpose": "assertionMethod",
            "verificationMethod": "did:key:z6Mkoe1KHk9aEJeTbWtC6AQTpX7FeUMNqSSVubVgY2M1X9st",
            "jws": ""
          }
        }),
        media_type="application/json",
        format="https://www.w3.org/TR/vc-data-model/"
    )])

tony_basic_message_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = tony_basic_message,
    frm = alice_did_new,
    to = nessus_invitation['from'],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

trust_ping_did_doc = json.loads(peer_did.resolve_peer_did(nessus_invitation["from"]))
trust_ping_endpoint = trust_ping_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(trust_ping_endpoint, headers=headers, json = json.loads(tony_basic_message_packed.packed_msg))
resp.text

'{}'

In [44]:
res = await poll_response()
res

Messages in Mediator queue: 1
message 6435eccafb5566e28c19e285 deleted


UnpackResult(message=Message(id='b0fdc53c-718b-48b6-af94-5df666d76b52', type='https://didcomm.org/issue-credential/3.0/issue-credential', body={}, frm='did:peer:2.Ez6LShroqrxP6vrwcyhyG7iDfZ9YpdNZzCVFEaaS8n9fkQQGL.Vz6MkkL7EaycmQTpdiodvEfQipKmjy2JCmHyYo4JEJWwXvCmE.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL25lc3N1cy10ZWNoLmlvOjkxMDAifQ', to=['did:peer:2.Ez6LSgNmZmqQFzvbVaE2mKYcF1TFrU89c88seVEwqL3FwENAc.Vz6MkqMG9iqP9iAk6B2qbLHs4yLavCXm54cPgW6GBdXPRT9gF.SeyJpZCI6IiNkaWRjb21tbWVzc2FnaW5nLTAiLCJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9tZWRpYXRvci5yb290c2lkLmNsb3VkIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2NhYWpIYlZ5RktxMlFYMXo0ZzU3OW1SQ1JBem1Fc1M0dENlcGFmS2laYko5LlZ6Nk1rcXl2ZkJ1blJmR1l4NXhqYkVvS2FYRVY0bmUzV1hEV0RVc1JLZHBFeDFQN3cuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyMWxaR2xoZEc5eUxuSnZiM1J6YVdRdVkyeHZkV1FpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIl19'], created_time=None, expires_time=None, from_prior=None, please_ack=None, ack=None, thid='1542eb12-32a6-414f-a983-18b2f5dff080',

In [45]:
res.message.attachments[0].data.json

{'type': ['VerifiableCredential', 'Passport'],
 '@context': ['https://www.w3.org/2018/credentials/v1',
  'https://w3id.org/security/suites/jws-2020/v1'],
 'id': 'urn:uuid:1cece1f8-6a68-4361-8823-08802b56ede8',
 'issuer': 'did:key:z6Mkoe1KHk9aEJeTbWtC6AQTpX7FeUMNqSSVubVgY2M1X9st',
 'issuanceDate': '2023-04-06T10:58:22.435583926Z',
 'credentialSchema': {'id': 'class:nessus/schema/PassportSchema-draft-01.json',
  'type': 'JsonSchemaValidator2018'},
 'credentialSubject': {'id': 'did:key:z6Mkrea1ndAivx7y7iXVgSBK98BLYmBm7RPdfEtwRN3MSVfj',
  'givenName': 'Alex',
  'familyName': 'Andrei',
  'citizenship': 'US'},
 'proof': {'type': 'JsonWebSignature2020',
  'creator': 'did:key:z6Mkoe1KHk9aEJeTbWtC6AQTpX7FeUMNqSSVubVgY2M1X9st',
  'created': '2023-04-06T10:58:22Z',
  'proofPurpose': 'assertionMethod',
  'verificationMethod': 'did:key:z6Mkoe1KHk9aEJeTbWtC6AQTpX7FeUMNqSSVubVgY2M1X9st',
  'jws': ''}}

# output of the run is here 
http://nessus-tech.io:9100/dashboard/messages?walletName=Government&pconId=067bff35-8d7a-4e78-8546-c2e219e7b5f5